# guppy and hugr to qir conversion and submission to H2


This example shows how to convert guppy to qir which can be submitted directly to H1 and H2 device, emulator and syntax checker.

You need to install hugr-qir, guppy and pytket-quantinuum for this notebook to work.

### Current guppy features that can't be converted:
- loops with condition not known at compiletime
- functions returning qubit arrays
- RNG functions
- dynamic qubit allocation


In [1]:
# You can write your guppy directly in a notebook or in a separate file
from typing import no_type_check

from guppylang import guppy, qubit
from guppylang.std.builtins import result
from guppylang.std.quantum import h, measure


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    h(q0)
    h(q1)

    b0 = measure(q0)
    b1 = measure(q1)
    b2 = b0 ^ b1

    result("0", b2)

# Convert hugr to qir

By default, the function will automatically check the generated QIR to capture most of the issues that could happen.
This will show an error message with more details about the problem the check can be turned off using the keyword argument `validate_qir = False`

In [2]:
guppy_qir_bitcode_string = main.compile().to_qir_str()

In [3]:
# To get a human-readable LLVM assemly language string use the `hugr_to_qir` function with the keyword argument `output_format = OutputFormat.LLVM_IR`

guppy_qir = main.compile().to_qir_str()
print(guppy_qir)

; ModuleID = 'hugr-qir'
source_filename = "hugr-qir"
target datalayout = "e-m:e-i8:8:32-i16:16:32-i64:64-i128:128-n32:64-S128"
target triple = "aarch64-unknown-linux-gnu"

%Qubit = type opaque
%Result = type opaque

@0 = private unnamed_addr constant [2 x i8] c"0\00", align 1

define dso_local void @__hugr__.main.1() local_unnamed_addr #0 {
alloca_block:
  tail call void @__quantum__qis__phasedx__body(double 0x3FF921FB54442D18, double 0xBFF921FB54442D18, %Qubit* null)
  tail call void @__quantum__qis__rz__body(double 0x400921FB54442D18, %Qubit* null)
  tail call void @__quantum__qis__mz__body(%Qubit* null, %Result* null)
  %0 = tail call i1 @__quantum__qis__read_result__body(%Result* null)
  tail call void @__quantum__qis__phasedx__body(double 0x3FF921FB54442D18, double 0xBFF921FB54442D18, %Qubit* nonnull inttoptr (i64 1 to %Qubit*))
  tail call void @__quantum__qis__rz__body(double 0x400921FB54442D18, %Qubit* nonnull inttoptr (i64 1 to %Qubit*))
  tail call void @__quantum__qis__mz__b

### Loops in the program are unrolled automatically when possible, because backwards branching is not available on H Series.  This means that the QIR generated from programs containing loops can get quite long as shown here:

In [4]:
from typing import no_type_check

from guppylang import guppy


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    for _ in range(10):
        q3 = qubit()
        h(q3)
        b = measure(q3)
        if b:
            h(q0)

    result("0", measure(q0))
    result("1", measure(q1))

In [5]:
guppy_qir = main.compile().to_qir_str()
print(guppy_qir)

; ModuleID = 'hugr-qir'
source_filename = "hugr-qir"
target datalayout = "e-m:e-i8:8:32-i16:16:32-i64:64-i128:128-n32:64-S128"
target triple = "aarch64-unknown-linux-gnu"

%Qubit = type opaque
%Result = type opaque

@0 = private unnamed_addr constant [2 x i8] c"0\00", align 1
@1 = private unnamed_addr constant [2 x i8] c"1\00", align 1

define dso_local void @__hugr__.main.1() local_unnamed_addr #0 {
alloca_block:
  tail call void @__quantum__qis__phasedx__body(double 0x3FF921FB54442D18, double 0xBFF921FB54442D18, %Qubit* nonnull inttoptr (i64 2 to %Qubit*))
  tail call void @__quantum__qis__rz__body(double 0x400921FB54442D18, %Qubit* nonnull inttoptr (i64 2 to %Qubit*))
  tail call void @__quantum__qis__mz__body(%Qubit* nonnull inttoptr (i64 2 to %Qubit*), %Result* nonnull inttoptr (i64 2 to %Result*))
  %0 = tail call i1 @__quantum__qis__read_result__body(%Result* nonnull inttoptr (i64 2 to %Result*))
  br i1 %0, label %21, label %cond_384_case_0

cond_384_case_0:                    

### A similar example to the one above with a loop that terminates based on a measurement result inside the loop leads to the generation of QIR which is valid within the standard, but can't be executed on H-Series devices.

In [6]:
from typing import no_type_check

from guppylang import guppy


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    i = 0
    while i < 10:
        q3 = qubit()
        h(q3)
        b = measure(q3)
        if b:
            h(q0)
            i += 1

    result("0", measure(q0))
    result("1", measure(q1))

In [7]:
guppy_qir = main.compile().to_qir_str(validate_qir=False)
print(guppy_qir)

; ModuleID = 'hugr-qir'
source_filename = "hugr-qir"
target datalayout = "e-m:e-i8:8:32-i16:16:32-i64:64-i128:128-n32:64-S128"
target triple = "aarch64-unknown-linux-gnu"

%Qubit = type opaque
%Result = type opaque

@0 = private unnamed_addr constant [2 x i8] c"0\00", align 1
@1 = private unnamed_addr constant [2 x i8] c"1\00", align 1

define dso_local void @__hugr__.main.1() local_unnamed_addr #0 {
alloca_block:
  br label %cond_exit_182

cond_104_case_0:                                  ; preds = %cond_exit_182, %4
  %"65_0.0" = phi i64 [ %5, %4 ], [ %"15_0.0221", %cond_exit_182 ]
  %0 = icmp slt i64 %"65_0.0", 10
  br i1 %0, label %cond_exit_182, label %cond_132_case_1

cond_132_case_1:                                  ; preds = %cond_104_case_0
  tail call void @__quantum__qis__mz__body(%Qubit* null, %Result* null)
  %1 = tail call i1 @__quantum__qis__read_result__body(%Result* null)
  tail call void @__quantum__rt__bool_record_output(i1 %1, i8* getelementptr inbounds ([2 x i8], [

In [8]:
# this will fail because of the loop in the generated QIR

try:
    guppy_qir = main.compile().to_qir_str()
    print(guppy_qir)
except Exception as e:
    print("Validation failed as expected:")
    print(e)

Validation failed as expected:
Found loop in CFG containing the block: cond_exit_182


# Submission to the device via Nexus

The QIR generated can be submitted directly to Nexus. The python Nexus API is available via `pip install qnexus`. This requires a different QIR format for the submission, which can be generated from `compile_qir`.

In [9]:
import qnexus as qnx

qnx.login()

Already logged in. Tokens are valid.


In [10]:
import datetime

project = qnx.projects.get_or_create(name="QIR-Demonstration")
qnx.context.set_active_project(project)

qir_name = "HUGR-QIR"
jobname_suffix = datetime.datetime.now().strftime("%Y_%m_%d-%H-%M-%S")

In [11]:
# You can write your guppy directly in a notebook or in a separate file
from typing import no_type_check

from guppylang import guppy


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    h(q0)
    h(q1)

    b0 = measure(q0)
    b1 = measure(q1)
    b2 = b0 ^ b1

    result("0", b2)

In [ ]:
guppy_qir_bitcode = main.compile().to_qir_bytes()

In [13]:
qir_program_ref = qnx.qir.upload(qir=guppy_qir_bitcode, name=qir_name, project=project)

In [14]:
# Run on the H2-1 Syntax checker
device_name = "H2-1SC"

qnx.context.set_active_project(project)
config = qnx.QuantinuumConfig(device_name=device_name)

job_name = f"execution-job-qir-{qir_name}-{device_name}-{jobname_suffix}"
ref_execute_job = qnx.start_execute_job(
    programs=[qir_program_ref],
    n_shots=[10],
    backend_config=config,
    name=job_name,
)

In [ ]:
qnx.jobs.wait_for(ref_execute_job)

In [ ]:
qir_result = qnx.jobs.results(ref_execute_job)[0].download_result()
qir_result.get_counts()